In [1]:
from __future__ import division
from __future__ import print_function

import os, sys
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import random
import config
import torch.optim as optim
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
# NEURAL NETWORK MODULES/LAYERS
from dataset import Dataset
# METRICS CLASS FOR EVALUATION
from metrics import Metrics
# CONFIG PARSER
from config import parse_args
# TRAIN AND TEST HELPER FUNCTIONS
from trainer import Trainer
import glob
from tqdm import tqdm
from PIL import Image
import torchvision
from torchvision import models
# from tensorboardX import SummaryWriter
from torchsummary import summary
#import model_vgg16
# import model_vgg19
import model_resnet50
# import model_densenet121
# import model_inceptionv3

# MAIN BLOCK
# def main():
# global args
# args = parse_args()
# global logger


In [2]:
torch.__version__

'1.2.0'

In [3]:
class config():
    def __init__(self):
        self.data='./data'
        self.save='checkpoints/'
        self.expname='resnet50_holistic_375x500'
        self.expno=0
        self.pretrained_model='resnet50'
        self.num_classes=16
        # training arguments
        self.epochs=100000
        self.batchsize=4
        self.lr=0.0001
        self.wd=1e-4
        self.sparse=False
        self.optim='adam'
        # miscellaneous options
        self.seed=123
        self.cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.cuda else "cpu")
        self.pretrained_holistic=0

#         cuda_parser = parser.add_mutually_exclusive_group(required=False)
#         cuda', dest='cuda', action='store_true')
#         cuda_parser.add_argument('--no-cuda', dest='cuda', action='store_false')
#         parser.set_defaults(cuda=True)
        

In [4]:
args = config()
# writer = SummaryWriter(comment = args.expname)
args.batchsize

4

In [5]:

torch.manual_seed(args.seed)
random.seed(args.seed)

if args.cuda:
    torch.cuda.manual_seed(args.seed)
    torch.backends.cudnn.benchmark = True

if not os.path.exists(args.save):
    os.makedirs(args.save)



In [6]:
def create_optimizer(args, parameters):
    if args.optim=='adam':
        optimizer   = optim.Adam(parameters, lr=args.lr, weight_decay=args.wd)
    elif args.optim=='adagrad':
        optimizer   = optim.Adagrad(parameters, lr=args.lr, weight_decay=args.wd)
    elif args.optim=='sgd':
        optimizer   = optim.SGD(parameters, lr=args.lr, weight_decay=args.wd)
    elif args.optim == 'adadelta':
        optimizer = optim.Adadelta(parameters, lr=args.lr, weight_decay=args.wd)
    return optimizer

In [7]:
def create_optimizer(args, parameters):
    if args.optim=='adam':
        optimizer   = optim.Adam(parameters, lr=args.lr, weight_decay=args.wd)
    elif args.optim=='adagrad':
        optimizer   = optim.Adagrad(parameters, lr=args.lr, weight_decay=args.wd)
    elif args.optim=='sgd':
        optimizer   = optim.SGD(parameters, lr=args.lr, weight_decay=args.wd)
    elif args.optim == 'adadelta':
        optimizer = optim.Adadelta(parameters, lr=args.lr, weight_decay=args.wd)
    return optimizer

# initialize model, criterion/loss_function, optimizer
pretrained_resnet50 = models.resnet50(pretrained=True)

# Freeze training for all layers
# for child in pretrained_vgg16.children():
#     for param in child.parameters():
#         param.requires_grad = False
start_point = 0

if args.pretrained_holistic == 0 and args.expname == 'resnet50_holistic_375x500':
    model = model_resnet50.DocClassificationHolistic(args, pretrained_resnet50)
    optimizer = create_optimizer(args, list(model.parameters()))

    if os.path.exists('./checkpoints/{0}.pt'.format(args.expname)):
        print('model exists')
        checkpoint = torch.load('./checkpoints/{0}.pt'.format(args.expname))
        model.load_state_dict(checkpoint['model'])
        if args.cuda:
            model.cuda()
        #start_point = checkpoint['epoch']
        start_point = 0
        optimizer.load_state_dict(checkpoint['optim'])
elif args.pretrained_holistic == 0 and args.expname != 'resnet50_holistic':
    model = model_resnet50.DocClassificationHolistic(args, pretrained_resnet50)
    optimizer = create_optimizer(args, list(model.parameters()))

    if os.path.exists('./checkpoints/raw_resnet50_holistic_multiple_loss.pt'):
        checkpoint = torch.load('./checkpoints/raw_resnet50_holistic_multiple_loss.pt')
        model.load_state_dict(checkpoint['model'])
        if args.cuda:
            model.cuda()
        #start_point = checkpoint['epoch']
        start_point = 0
        optimizer.load_state_dict(checkpoint['optim'])

elif args.pretrained_holistic == 1:
    checkpoint_holistic = torch.load('./checkpoints/resnet50_holistic.pt')
    #resnet50 = model_resnet50.DocClassificationHolistic(args, pretrained_resnet50)
    pretrained_holistic_model = model_resnet50.DocClassificationHolistic(args, pretrained_resnet50)
    pretrained_holistic_model.load_state_dict(checkpoint_holistic['model'])

    model = model_resnet50.DocClassificationRest(args, pretrained_holistic_model)

    parameters = list(pretrained_holistic_model.parameters())

    if os.path.exists('./checkpoints/%s.pt'%(args.expname)):
        checkpoint_ensemble = torch.load('./checkpoints/%s.pt'%(args.expname))
        model.load_state_dict(checkpoint_ensemble['model'])
        if args.cuda:
            model.cuda()

        start_point = checkpoint_ensemble['epoch']
        optimizer = create_optimizer(args, parameters)
        optimizer.load_state_dict(checkpoint_ensemble['optim'])
    else:
        optimizer = create_optimizer(args, parameters)
        #optimizer.load_state_dict(checkpoint_holistic['optim'])


criterion = nn.CrossEntropyLoss(reduction='mean')


if args.cuda:
    model.cuda(), criterion.cuda()


metrics = Metrics(args.num_classes)

# create trainer object for training and testing
trainer = Trainer(args, model, criterion, optimizer)

model exists


In [8]:
print(model)
for parameter, name in zip(model.parameters(), model.named_parameters()):
     print("Param Name: " + name[0] + " Requires Training: " + str(parameter.requires_grad))
print(optimizer)

DocClassificationHolistic(
  (pretrained_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsamp

In [9]:
# train_dir = glob.glob(os.path.join(args.data,'train/holistic/*.jpg'))
# dev_dir = glob.glob(os.path.join(args.data,'val/holistic/*.jpg'))
# test_dir = glob.glob(os.path.join(args.data,'test/holistic/*.jpg'))

# print(len(train_dir), len(dev_dir), len(test_dir))

# 'data/labels/train.csv'
# train_dataset = Dataset('data/stratified/rvl-cdip/labels/train.csv',model_type=args.pretrained_model)
# dev_dataset = Dataset('data/stratified/rvl-cdip/labels/val.csv', model_type=args.pretrained_model)

test_dataset = Dataset('data/stratified/rvl-cdip/labels/test.csv', model_type=args.pretrained_model)

# print('==> Size of train data   : %d ' % len(train_dataset))
# print('==> Size of val data   : %d ' % len(dev_dataset))
print('==> Size of test data   : %d ' % len(test_dataset))

# train_idx = list(np.arange(len(train_dataset)))
# dev_idx = list(np.arange(len(dev_dataset)))
test_idx = list(np.arange(len(test_dataset)))

best = float('inf')
columns = ['ExpName','ExpNo', 'Epoch', 'Loss','Accuracy']
results = []
early_stop_count = 0

==> Size of test data   : 15999 


In [10]:
bh,h,f,l,r,lab = test_dataset[np.random.randint(0,16000,64)]
batch_holistic = torch.unsqueeze(bh , dim=1).repeat(1,3,1,1).float().cuda()
# # bh = np.array(bh.detach().cpu())
print(bh.shape, batch_holistic.shape)
# batch_header = torch.unsqueeze(h, dim=1).repeat(1,3,1,1).float().cuda()
# batch_footer = torch.unsqueeze(f, dim=1).repeat(1,3,1,1).float().cuda()
# batch_left = torch.unsqueeze(l, dim=1).repeat(1,3,1,1).float().cuda()
# batch_right = torch.unsqueeze(r, dim=1).repeat(1,3,1,1).float().cuda()

# all_data = [batch_holistic,batch_header,batch_footer,batch_left,batch_right]
# writer.add_graph(model, input_to_model=all_data)
# writer.flush()
summary(model, batch_holistic.shape[1:])

torch.Size([64, 500, 375]) torch.Size([64, 3, 500, 375])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 250, 188]           9,408
       BatchNorm2d-2         [-1, 64, 250, 188]             128
              ReLU-3         [-1, 64, 250, 188]               0
         MaxPool2d-4          [-1, 64, 125, 94]               0
            Conv2d-5          [-1, 64, 125, 94]           4,096
       BatchNorm2d-6          [-1, 64, 125, 94]             128
              ReLU-7          [-1, 64, 125, 94]               0
            Conv2d-8          [-1, 64, 125, 94]          36,864
       BatchNorm2d-9          [-1, 64, 125, 94]             128
             ReLU-10          [-1, 64, 125, 94]               0
           Conv2d-11         [-1, 256, 125, 94]          16,384
      BatchNorm2d-12         [-1, 256, 125, 94]             512
           Conv2d-13         [-1, 256, 125, 94

In [11]:
# from matplotlib import pyplot as plt
# def plot_data(x,ax):
# #     x = x.reshape((size,size))
    
#     ax.imshow(x, cmap='gray')
# #     if y is not None:
# #         ax.scatter(y[0::2] , y[1::2] , marker='x', s=10)
# def plot_images(x,batch_size=64):
#     fig = plt.figure(figsize=(50,50))
#     fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.01, wspace=0.01)
#     for i in range(batch_size):
#         ax = fig.add_subplot(8, 8, i + 1, xticks=[], yticks=[])
#     #     print(y['classes'][i])
#     #     print(np.argmax(c[i]))
#         plot_data(x[i], ax)
# plot_images(bh)
# print(bh.shape,bh.min(),bh.max())
# print(lab)
# # display(plot_images(np.squeeze(seg[0],axis=-1)))

In [12]:
batch_test_idxs = []
counter = 0

while counter<len(test_dataset):
    batch_test_idx = np.array(list(np.arange(counter, counter+args.batchsize)))
    batch_test_idxs.append(batch_test_idx)
    counter+=args.batchsize
batch_test_idxs

test_acc = []
test_predictions = []
test_labels = []


# ----------------- Test ---------------------
for batch_test_idx in tqdm(batch_test_idxs, desc='Test batches..'):
#     print(batch_test_idx)
    try:
        test_batch_holistic, \
                test_batch_header, \
                test_batch_footer, \
                test_batch_left_body, \
                test_batch_right_body, \
                test_batch_labels = test_dataset[batch_test_idx]

        if args.pretrained_holistic == 0 and 'multiple_loss' not in args.expname: # singleloss
            test_batch_loss, test_batch_predictions, test_batch_labels = trainer.test_holistic(test_batch_holistic, test_batch_labels)
        elif args.pretrained_holistic == 0 and 'multiple_loss' in args.expname: # multipleloss
            test_batch_loss, test_batch_predictions, test_batch_labels = trainer.test_rest_multiple_loss(test_batch_holistic, \
                                                                                    test_batch_header, \
                                                                                    test_batch_footer, \
                                                                                    test_batch_left_body, \
                                                                                    test_batch_right_body, \
                                                                                    test_batch_labels)
        elif args.pretrained_holistic == 1:
            test_batch_loss, test_batch_predictions, test_batch_labels = trainer.test_rest(test_batch_holistic, \
                                    test_batch_header, \
                                    test_batch_footer, \
                                    test_batch_left_body, \
                                    test_batch_right_body, \
                                    test_batch_labels)
            
        elif args.pretrained_holistic == 2:
            test_batch_loss, test_batch_predictions, test_batch_labels = trainer.test_ensemble(test_batch_holistic, \
                                    test_batch_header, \
                                    test_batch_footer, \
                                    test_batch_left_body, \
                                    test_batch_right_body, \
                                    test_batch_labels)
    #         test_predictions.append(test_batch_predictions)
    #         test_labels.append(test_batch_labels)
    #         test_loss = test_loss + test_batch_loss

        test_accuracy = metrics.accuracy(test_batch_predictions, test_batch_labels)
    #     print(test_accuracy)
        test_acc.append(test_accuracy)
    except Exception as ex:
        print(ex)
        



Test batches..: 100%|██████████| 4000/4000 [03:45<00:00, 17.74it/s]

15999


In [18]:
'Accuracy of {0}: {1} %'.format(args.expname, np.round(np.mean(test_acc)*100,2))


'Accuracy of resnet50_holistic_375x500: 86.7 %'

In [17]:
test_acc

[1.0,
 1.0,
 1.0,
 0.75,
 1.0,
 1.0,
 0.75,
 1.0,
 0.75,
 1.0,
 0.75,
 0.75,
 0.75,
 1.0,
 0.75,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.75,
 1.0,
 1.0,
 1.0,
 0.75,
 1.0,
 1.0,
 0.75,
 1.0,
 1.0,
 0.75,
 0.75,
 0.5,
 0.5,
 1.0,
 0.75,
 1.0,
 1.0,
 0.75,
 0.75,
 1.0,
 1.0,
 1.0,
 1.0,
 0.75,
 1.0,
 0.75,
 1.0,
 0.75,
 1.0,
 1.0,
 0.75,
 1.0,
 1.0,
 0.75,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.75,
 1.0,
 0.75,
 1.0,
 0.75,
 0.5,
 1.0,
 1.0,
 1.0,
 0.75,
 1.0,
 1.0,
 1.0,
 0.75,
 1.0,
 0.75,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.75,
 1.0,
 1.0,
 0.75,
 0.75,
 0.5,
 0.75,
 0.75,
 1.0,
 1.0,
 0.75,
 1.0,
 1.0,
 0.75,
 1.0,
 1.0,
 1.0,
 0.75,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.75,
 1.0,
 1.0,
 0.75,
 1.0,
 0.75,
 1.0,
 0.75,
 1.0,
 0.75,
 0.75,
 0.75,
 0.25,
 0.75,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5,
 1.0,
 0.75,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.75,
 1.0,
 0.75,
 1.0,
 1.0,
 1.0,
 1.0,
 0.75,
 1.0,
 1.0,
 0.75,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.7